CROSS VALIDATION (PUNTO 3)

In [ ]:
#importamos todas las liberías necesarias. 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import balanced_accuracy_score, accuracy_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
import pickle


1. CARGAMOS TODOS LOS DATOS

In [ ]:
# Cargar los datos (reemplaza con tu archivo específico)
available_data_path = f"attrition_availabledata_03.csv"
df = pd.read_csv(available_data_path)

2. ELIMINAMOS TODOS LOS DATOS IRRELEVANTES 

In [ ]:
# Eliminar columnas irrelevantes
drop_columns = ["EmployeeID", "Over18", "StandardHours", "EmployeeCount"]
df = df.drop(columns=drop_columns, errors='ignore')

3. CONVERTIMOS TODAS LAS VARIABLES A TIPO NUMÉRICO Y SEPARAMOS SEGÚN CARACTERÍSTICAS Y VARIABLE OBJETIVO

In [ ]:
# Convertir variables categóricas a numéricas usando Label Encoding
categorical_columns = df.select_dtypes(include=['object']).columns
label_encoders = {}

for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le  # Guardar para futuras conversiones si es necesario

# Separar en características (X) y variable objetivo (y)
X = df.drop(columns=["Attrition"])
y = df["Attrition"]

4. DIVIDIMOS EN SECTORES Y NORMALIZAMOS TODOS LOS DATOS 

In [ ]:

# Dividir en Train (2/3) y Test (1/3)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3, random_state=3, stratify=y)

# Normalización (opcional, útil para algunos modelos como KNN)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Guardar el scaler para futuros usos
with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

# Evaluación INNER: K-Fold Cross Validation (k=5)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=3)

print("Datos preparados: Train (2/3), Test (1/3), y Cross-Validation lista.")


EVALCUACIÓN OUTER: ESTIMACION DE RENDIMIENTO CON UN MODELO BASE

In [ ]:
# Entrenar modelo base en Train
model = DecisionTreeClassifier(random_state=3)
model.fit(X_train_scaled, y_train)

In [ ]:
# Predecir en el conjunto de Test (Outer Evaluation)
y_pred = model.predict(X_test_scaled)

In [ ]:
# 🔹 Cálculo de métricas
balanced_acc = balanced_accuracy_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

In [ ]:
# Matriz de confusión para obtener TPR y TNR
conf_matrix = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = conf_matrix.ravel()


In [ ]:
# True Positive Rate (TPR) y True Negative Rate (TNR)
TPR = tp / (tp + fn)
TNR = tn / (tn + fp)

# 📌 Mostrar resultados
print(f"Balanced Accuracy: {balanced_acc:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"True Positive Rate (TPR): {TPR:.4f}")
print(f"True Negative Rate (TNR): {TNR:.4f}")
print(f"Matriz de Confusión:\n{conf_matrix}")